## Homework 3 MAE-207

## Connect to ODrive

In [1]:
import time
from multiprocessing import Process
from IPython.display import clear_output
import numpy as np
import matplotlib as mpl

# import sympy
from sympy import symbols, pprint
from sympy import sin, cos, asin, acos, atan, pi, sqrt
from sympy import Matrix
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from sympy import lambdify
# import numpy as np

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import style

from IPython import display # for the animation
# import matplotlib as mpl
# mpl.use('Qt5Agg')
# plt.ion()
import os
import matplotlib.pyplot as plt
# plt.ion()

# for the symbolic manipulation of jacobian
import sympy as sp
# from sympy import symbols
#from sympy import sin, cos, asin, acos, pi, atan2, sqrt
from sympy.utilities.lambdify import lambdify
#from sympy import Matrix

from scipy.optimize import minimize
from scipy.optimize import fsolve

import time

import odrive
from odrive.utils import *
import odrive.enums
%matplotlib qt

odrv0 = odrive.find_any()
if odrv0 is not None:
    print('Connected!')
    print('Odrive serial {}'.format(odrv0.serial_number))
    
    m0 = odrv0.axis0.motor.is_calibrated
    m1 = odrv0.axis1.motor.is_calibrated
    
    print('Motor 0 calibrated: {}'.format(m0))
    print('Motor 1 calibrated: {}'.format(m1))    
    
else:
    print('Not connected')
    

odrv0.serial_number

## Homework 3 MAE-207

## Connect to ODrive

odrv0 = odrive.find_any()

print('Odrive serial {}'.format(odrv0.serial_number))

### JJ_Odrv0=20863883304E
### SS_Odrv0=306139573235

# odrv0 = odrive.find_any()
# if odrv0 is not None:
#     print('Connected!')
#     print('Odrive serial {}'.format(odrv0.serial_number))
    
#     m0 = odrv0.axis0.motor.is_calibrated
#     m1 = odrv0.axis1.motor.is_calibrated
    
#     print('Motor 0 calibrated: {}'.format(m0))
#     print('Motor 1 calibrated: {}'.format(m1))    
    
# else:
#     print('Not connected')
    

In [240]:
def connect_odrive(serial):
    odrv0 = odrive.find_any(serial_number = serial, timeout = 20)
    if odrv0 is not None:
        print('Connected!')
        print('Target serial {} \t Odrive serial {}'.format(serial, odrv0.serial_number))
        
        m0 = odrv0.axis0.motor.is_calibrated
        m1 = odrv0.axis1.motor.is_calibrated

        print('Motor 0 calibrated: {}'.format(m0))
        print('Motor 1 calibrated: {}'.format(m1))    
    else:
        print("ODrive ({}) not found".format(serial))
    return odrv0
    
JJ_odrv0 = connect_odrive('20863883304E')
SS_odrv0 = connect_odrive('306139573235')


Connected!
Target serial 20863883304E 	 Odrive serial 35760845828174
Motor 0 calibrated: True
Motor 1 calibrated: True
Connected!
Target serial 306139573235 	 Odrive serial 53194131976757
Motor 0 calibrated: True
Motor 1 calibrated: True


In [241]:
# Setting the velocity and calibration current limit
JJ_odrv0.axis0.controller.config.vel_limit=2000000
JJ_odrv0.axis1.controller.config.vel_limit=2000000
JJ_odrv0.axis0.motor.config.calibration_current=20.0
JJ_odrv0.axis1.motor.config.calibration_current=20.0

SS_odrv0.axis0.controller.config.vel_limit=2000000
SS_odrv0.axis1.controller.config.vel_limit=2000000
SS_odrv0.axis0.motor.config.calibration_current=20.0
SS_odrv0.axis1.motor.config.calibration_current=20.0


#### Calibrating the Motors

In [175]:
SS_odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_FULL_CALIBRATION_SEQUENCE
SS_odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_FULL_CALIBRATION_SEQUENCE

JJ_odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_FULL_CALIBRATION_SEQUENCE
JJ_odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_FULL_CALIBRATION_SEQUENCE

time.sleep(15)

print('\t Motor 0 calibration result: {} \r\n'.format(SS_odrv0.axis0.motor.is_calibrated), 
      '\t Motor 1 calibration result: {}'.format(SS_odrv0.axis1.motor.is_calibrated))
print('\t Motor 0 calibration result: {} \r\n'.format(JJ_odrv0.axis0.motor.is_calibrated), 
      '\t Motor 1 calibration result: {}'.format(JJ_odrv0.axis1.motor.is_calibrated))

SS_odrv0.axis0.motor.config.pre_calibrated = True
SS_odrv0.axis1.motor.config.pre_calibrated = True

JJ_odrv0.axis0.motor.config.pre_calibrated = True
JJ_odrv0.axis1.motor.config.pre_calibrated = True

SS_odrv0.save_configuration()
JJ_odrv0.save_configuration()


ChannelBrokenException: 

SS_odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
SS_odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL

SS_odrv0.axis0.controller.set_pos_setpoint(0,0,0)
SS_odrv0.axis1.controller.set_pos_setpoint(0,0,0)

In [6]:
# Function to setup home position
def autohome(odrv0):
    print('Bring motors to home and press ENTER')
    input("<enter>")
    R_pos1=odrv0.axis0.encoder.pos_estimate
    L_pos1=odrv0.axis1.encoder.pos_estimate
    print('Bring motors to 90-90 and press ENTER')
    input("<enter>")
    R_pos2=odrv0.axis0.encoder.pos_estimate
    L_pos2=odrv0.axis1.encoder.pos_estimate
    ML_para=(90/(L_pos2-L_pos1))
    MR_para=((-90)/(R_pos2-R_pos1))
    print(ML_para)
    print(MR_para)
    print(L_pos2, L_pos1, R_pos2, R_pos1)
    print("Done")
    return ML_para,MR_para,L_pos1,R_pos1

autohome(JJ_odrv0)

autohome(SS_odrv0)

In [9]:
def idle_all():
    JJ_odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_IDLE
    JJ_odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_IDLE
    SS_odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_IDLE
    SS_odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_IDLE
    

In [10]:
# Function to make the motors return to idle mode 
def motor_idle(odrv0): 
    odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_IDLE
    odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_IDLE

# Defining the Symbols
(theta_l, 
 theta_r,
 l1, 
 l2,
 w) = symbols(""" theta_l
                         theta_r
                         l1 
                         l2
                         w """ , real = True)

#### Forward Kinematics definition

A = 2*l2*l1*sin(theta_l)- 2*l1*l2*sin(theta_r)
B = 2*l2*w - 2*l1*l2*cos(theta_r) + 2*l2*l1*cos(theta_l)
C = 2*(l1**2) + w**2 - 2*l1*l1*sin(theta_l)*sin(theta_r) - 2*l1*w*cos(theta_r) + 2*l1*w*cos(theta_l) - 2*l1*l1*cos(theta_l)*cos(theta_r)

theta3_1 = 2*atan((A + sqrt(A**2 + B**2 - C**2))/(B-C))
theta3_2 = 2*atan((A - sqrt(A**2 + B**2 - C**2))/(B-C))

xc_1 = l1*cos(theta_l) + l2*cos(theta3_1) + w/2
xc_1
xc_2 = l1*cos(theta_l) + l2*cos(theta3_2) + w/2
xc_2

yc_1 = l1*sin(theta_l) + l2*sin(theta3_1)
yc_1
yc_2 = l1*sin(theta_l) + l2*sin(theta3_2)
yc_2
#sympy.simplify(theta3_1)
#sympy.simplify(theta3_2)

FK1 = Matrix([[xc_1],
             [yc_1]])
FK2 = Matrix([[xc_2],
             [yc_2]])

FK1_fast = lambdify((theta_l,theta_r,l1,l2,w),FK1)
FK2_fast = lambdify((theta_l,theta_r,l1,l2,w),FK2)

def enc_pos(L,R,cal_para):
    L_pos=(np.rad2deg(L )/cal_para[0])+cal_para[2]
    R_pos=((np.rad2deg(R )-180)/cal_para[1])+cal_para[3]
    return L_pos,R_pos

def get_angles(odrv0,cal_para):
    theta_R=((odrv0.axis0.encoder.pos_estimate)-cal_para[3])*(cal_para[1])+180
    theta_L=((odrv0.axis1.encoder.pos_estimate)-cal_para[2])*(cal_para[0])

#     theta_R=(np.around(odrv0.axis0.encoder.pos_estimate)-cal_para[3])*(cal_para[1])+180
#     theta_L=((np.around(odrv0.axis1.encoder.pos_estimate)-cal_para[2])*cal_para[0])
        
    return np.deg2rad(theta_L),np.deg2rad(theta_R)

def get_vel(odrv0):
    vel_R=-(odrv0.axis0.encoder.vel_estimate/(2048*4))*(2*np.pi)
    vel_L=-(odrv0.axis1.encoder.vel_estimate/(2048*4))*(2*np.pi)
    
    return np.array([[vel_L],[vel_R]])

J1 = FK1.jacobian([theta_l,theta_r])
J1_fast = lambdify((theta_l,theta_r,l1,l2,w),J1)
# J_rtheta = FK_rtheta.jacobian([theta_l,theta_r])

link1_length = 0.09;                  # m 
link2_length = 0.16;                  # m
w_length = 0.07; 
x_eq = 0
y_eq = 0.12

theta=[]
cal_para_JJ=autohome(JJ_odrv0)
while True:
    theta_R=(np.around(odrv0.axis0.encoder.pos_estimate)-cal_para_JJ[3])*(cal_para_JJ[1])+180
    theta_L=((np.around(odrv0.axis1.encoder.pos_estimate)-cal_para_JJ[2])*cal_para_JJ[0])
    print(theta_L, theta_R)
    clear_output()
#     os.system('cls')

    theta.append([theta_R,theta_L])
    
Angle_value = np.asarray(theta)
np.save('Angle_Workspace',Angle_value)

plt.scatter(*zip(*theta))

plt.scatter(*zip(*theta))

#### Function to implement Inverse Kinematics using Minimize function

l1 = 0.09;                   
l2 = 0.16;                 
w = 0.07;  
def IK_5_link(x, y, l1 = l1, l2 = l2, w = w):
    
    def leg_wide(var):
        return np.linalg.norm([var[1] - np.pi, var[0]])
    
    def x_constraint_equation(var):
        return l1**2 - l2**2 + (x - w/2)**2 + y**2 - 2*l1*(y*np.sin(var[0]) + (x - w/2)*np.cos(var[0]))

    def y_constraint_equation(var):
        return l1**2 - l2**2 + (x + w/2)**2 + y**2 - 2*l1*(y*np.sin(var[1]) + (x + w/2)*np.cos(var[1]))

    
    res = minimize(leg_wide, (0.1, 9*np.pi/10), method="SLSQP", constraints= ({"type": "eq", "fun": x_constraint_equation}, 
                                                                               {"type": "eq", "fun": y_constraint_equation}))
    
    return (res, np.linalg.norm([x_constraint_equation(res.x), y_constraint_equation(res.x)]))

#### Code to implement the GAITs provided to us. 
This function returns a list of $\theta_L$ and $\theta_R$ for a giveb gait trajctory. These are later used to make the leg move in the desired trajectory

def get_trajectory(x,y,d,T,N):
    
    theta_points = []
    # Stance
    z = np.polyfit([x[0],x[1]],[y[0],y[1]],1)
    p = np.poly1d(z)
    n_stance = (np.rint(N*d))
    n_stance = n_stance.astype(int)
    x_stancepoints = np.linspace(x[0],x[1],n_stance+1)
    y_stancepoints = p(x_stancepoints)
    
    # Swing
    z = np.polyfit(x,y,2)
    p = np.poly1d(z)
    n_swing = np.rint((N*(1-d)))
    n_swing = n_swing.astype(int)
    x_swingpoints = np.linspace(x[1],x[0],n_swing+1)
    y_swingpoints = p(x_swingpoints)
    
    x_points = np.append(x_stancepoints,x_swingpoints[1:])
    y_points = np.append(y_stancepoints,y_swingpoints[1:])
    
    t_points = np.linspace(0,T,N+1)
    
    for i in range(x_points.shape[0]):
        
        res = IK_5_link(x_points[i],y_points[i])
        theta_points.append([res[0].x[1],res[0].x[0]])
    
    return theta_points,t_points

x = np.array([0.11,-0.11,0])
y = np.array([0.12+0.05,0.12+0.05,0.05+0.05])

data1, t1_points = get_trajectory(x,y,0.5,0.25,100)

x = np.array([0.11-0.05,-0.08-0.05,0.05-0.05])
y = np.array([0.14+0.05,0.1+0.05,0.05+0.05])

data2, t2_points = get_trajectory(x,y,0.75,0.5,100)

x = np.array([0.08,-0.08,0])
y = np.array([0.1+0.05,0.1+0.05,0.04+0.05])

data3, t3_points = get_trajectory(x,y,0.5,1.5,200)

#### Store the $\theta_L$ and $\theta_R$ to a numpy file to plot the data

np.save('theta_values_Data_2',data1)
np.save('theta_data3',data2)
np.save('theta_data4',data3)

#### Set Motor PID 

In [246]:
JJ_odrv0.axis0.controller.config.vel_gain = 16 / 10000.0
JJ_odrv0.axis1.controller.config.vel_gain = 16 / 10000.0
JJ_odrv0.axis1.controller.config.pos_gain=44
JJ_odrv0.axis1.controller.config.pos_gain=44

JJ_odrv0.axis0.controller.config.vel_integrator_gain = 0.001 #/ 10000.0
JJ_odrv0.axis1.controller.config.vel_integrator_gain = 0.001 #5.0 / 10000.0

In [247]:
SS_odrv0.axis0.controller.config.vel_gain = 16 / 10000.0
SS_odrv0.axis1.controller.config.vel_gain = 16 / 10000.0
SS_odrv0.axis1.controller.config.pos_gain=44
SS_odrv0.axis1.controller.config.pos_gain=44

SS_odrv0.axis0.controller.config.vel_integrator_gain = 0.001 #/ 10000.0
SS_odrv0.axis1.controller.config.vel_integrator_gain = 0.001 #5.0 / 10000.0

In [248]:
print('Motors JJ Homing')
cal_para_JJ=autohome(JJ_odrv0)
print('Motors SS Homing')
cal_para_SS=autohome(SS_odrv0)

Motors JJ Homing
Bring motors to home and press ENTER


<enter> 


Bring motors to 90-90 and press ENTER


<enter> 


-0.043504529154991436
-0.045067601402103155
-1075.0001220703125 993.75 -906.000244140625 -2903.000244140625
Done
Motors SS Homing
Bring motors to home and press ENTER


<enter> 


Bring motors to 90-90 and press ENTER


<enter> 


-0.04189669845287713
-0.0431402507527075
819.609375 2967.75 1988.90625 -97.3125
Done


JJ_odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
JJ_odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL

JJ_odrv0.axis0.controller.set_pos_setpoint(0,0,0)
JJ_odrv0.axis1.controller.set_pos_setpoint(0,0,0)

# odrv0=SS_odrv0
theta_Gait=[]
JJ_odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
JJ_odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
SS_odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
SS_odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL

a=np.load('data_L_leg.npy') # Left Leg 
b=np.load('data_R_leg.npy') # Right Leg

# Generic Hopping
start_time = time.time()
# your code
cpr= 2048
t_step = 0.005
w_ra = 0
w_la = 0

w_rb = 0
w_lb = 0
for k in range(0,2):
    for i in range(0,len(a)):
        start_time2 = time.time()
        L_posJ=(np.rad2deg(a[i][1] )/cal_para_JJ[0])+cal_para_JJ[2]
        R_posJ=((np.rad2deg(a[i][0] )-180)/cal_para_JJ[1])+cal_para_JJ[3]
        
        L_posS=(np.rad2deg(b[i][1] )/cal_para_SS[0])+cal_para_SS[2]
        R_posS=((np.rad2deg(b[i][0] )-180)/cal_para_SS[1])+cal_para_SS[3]

        if (i-1)>0:
            w_ra = ((a[i][0] - a[i-1][0])/(t_step*2*np.pi) )*cpr
            w_la = ((a[i][1] - a[i-1][1])/(t_step*2*np.pi) )*cpr
            
            w_rb = ((b[i][0] - b[i-1][0])/(t_step*2*np.pi) )*cpr
            w_lb = ((b[i][1] - b[i-1][1])/(t_step*2*np.pi) )*cpr
            
        SS_odrv0.axis0.controller.set_pos_setpoint(R_posS,1*w_rb,0)
        SS_odrv0.axis1.controller.set_pos_setpoint(L_posS,1*w_lb,0)
        
        JJ_odrv0.axis0.controller.set_pos_setpoint(R_posJ,1*w_ra,0)
        JJ_odrv0.axis1.controller.set_pos_setpoint(L_posJ,1*w_la,0)
        
        while (time.time() - start_time2 < 0.2*t_step):
            pass
            #clear_output()
#         time.sleep(0.1)
#         theta_R=(np.around(JJ_odrv0.axis0.encoder.pos_estimate)-cal_para_JJ[3])*(cal_para_JJ[1])+180
#         theta_L=((np.around(JJ_odrv0.axis1.encoder.pos_estimate)-cal_para_JJ[2])*cal_para_JJ[0])
#         theta_Gait.append([theta_R,theta_L])
# motor_idle()    
elapsed_time = time.time() - start_time
print(elapsed_time)

# Right leg cycle (motor set SS)
def right_leg_Cycle(JJ_odrv0):
    JJ_odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
    JJ_odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
    start_time = time.time()
    a=np.load('L_leg.npy')
    # your code
    cpr= 2048
    t_step = 0.005
    w_ra = 0
    w_la = 0

    w_rb = 0
    w_lb = 0
#     for k in range(0,10):

    for i in range(0,len(a)):
        start_time2 = time.time()
        L_posJ=(np.rad2deg(a[i][1] )/cal_para_JJ[0])+cal_para_JJ[2]
        R_posJ=((np.rad2deg(a[i][0] )-180)/cal_para_JJ[1])+cal_para_JJ[3]
#         print(L_posJ)
#         L_posS=(np.rad2deg(b[i][1] )/cal_para_SS[0])+cal_para_SS[2]
#         R_posS=((np.rad2deg(b[i][0] )-180)/cal_para_SS[1])+cal_para_SS[3]

        if (i-1)>0:
            w_ra = ((a[i][0] - a[i-1][0])/(t_step*2*np.pi) )*cpr
            w_la = ((a[i][1] - a[i-1][1])/(t_step*2*np.pi) )*cpr

#             w_rb = ((b[i][0] - b[i-1][0])/(t_step*2*np.pi) )*cpr
#             w_lb = ((b[i][1] - b[i-1][1])/(t_step*2*np.pi) )*cpr

#         SS_odrv0.axis0.controller.set_pos_setpoint(R_posS,1*w_rb,0)
#         SS_odrv0.axis1.controller.set_pos_setpoint(L_posS,1*w_lb,0)

        JJ_odrv0.axis0.controller.set_pos_setpoint(R_posJ,1*w_ra,0)
        JJ_odrv0.axis1.controller.set_pos_setpoint(L_posJ,1*w_la,0)

        while (time.time() - start_time2 < 0.2*t_step):
            pass
# motor_idle()    
    elapsed_time = time.time() - start_time
#     print(elapsed_time)

# Right leg cycle (motor set SS)
def left_leg_Cycle(SS_odrv0):
    SS_odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
    SS_odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
    start_time = time.time()
    a=np.load('R_leg.npy')
    # your code
    cpr= 2048
    t_step = 0.005
    w_ra = 0
    w_la = 0

    w_rb = 0
    w_lb = 0
#     for k in range(0,10):

    for i in range(0,len(a)):
        start_time2 = time.time()
#         L_posJ=(np.rad2deg(a[i][1] )/cal_para_JJ[0])+cal_para_JJ[2]
#         R_posJ=((np.rad2deg(a[i][0] )-180)/cal_para_JJ[1])+cal_para_JJ[3]
#         print(L_posJ)
        L_posS=(np.rad2deg(a[i][1] )/cal_para_SS[0])+cal_para_SS[2]
        R_posS=((np.rad2deg(a[i][0] )-180)/cal_para_SS[1])+cal_para_SS[3]

        if (i-1)>0:
#             w_ra = ((a[i][0] - a[i-1][0])/(t_step*2*np.pi) )*cpr
#             w_la = ((a[i][1] - a[i-1][1])/(t_step*2*np.pi) )*cpr

            w_rb = ((a[i][0] - a[i-1][0])/(t_step*2*np.pi) )*cpr
            w_lb = ((a[i][1] - a[i-1][1])/(t_step*2*np.pi) )*cpr

        SS_odrv0.axis0.controller.set_pos_setpoint(R_posS,1*w_rb,0)
        SS_odrv0.axis1.controller.set_pos_setpoint(L_posS,1*w_lb,0)

#         JJ_odrv0.axis0.controller.set_pos_setpoint(R_posJ,1*w_ra,0)
#         JJ_odrv0.axis1.controller.set_pos_setpoint(L_posJ,1*w_la,0)

        while (time.time() - start_time2 < 0.2*t_step):
            pass
# motor_idle()    
    elapsed_time = time.time() - start_time
#     print(elapsed_time)

# Cartesian Control Right set
def Stiff_SpringR(odrv0,cal_para):
    odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
    odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL

    # axis.controller.config.control_mode = CTRL_MODE_CURRENT_CONTROL

    x_eq=-0.11
    y_eq=0.12

    kx = 0  # N/m
    ky = 200 # N/m
    kT = 0.0285 # Nm/A
    c  = 1  #1.5 # Nm/rad
    Imax=25    #A
    res = IK_5_link(x_eq, y_eq)
    L_eq,R_eq=enc_pos(res[0].x[0],res[0].x[1],cal_para)

    odrv0.axis0.controller.set_pos_setpoint(R_eq,0,0)
    odrv0.axis1.controller.set_pos_setpoint(L_eq,0,0)
    time.sleep(1)
    t_end = time.time() + 5

    while time.time() < t_end:

        theta_l, theta_r = get_angles(odrv0,cal_para)
        vel = get_vel(odrv0)

        fk_xy = FK1_fast(theta_l,theta_r,link1_length,link2_length,w_length)
        J_xy = J1_fast(theta_l,theta_r,link1_length,link2_length,w_length)
        x = fk_xy[0]
        y = fk_xy[1]
        mat_xy = np.array([[x[0]-x_eq],
                           [y[0]-y_eq]])
        mat_k = np.array([[-kx,0],
                          [0,-ky]])
        I1 = (J_xy.T @ mat_k @ mat_xy)/kT
        I = I1 -c*(vel)
        if(I[0]>Imax):
            I[0]=Imax
        elif(I[0]<-Imax):
            I[0]=-Imax

        if(I[1]>Imax):
            I[1]=Imax
        elif(I[1]<-Imax):
            I[1]=-Imax

        odrv0.axis0.controller.set_current_setpoint(-I[1])
        odrv0.axis1.controller.set_current_setpoint(-I[0])
        print(I1)
        clear_output()
    motor_idle(odrv0)    

# Cartesian Control
def Stiff_SpringL(odrv0,cal_para,x_eq,y_eq):
    odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
    odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL

    # axis.controller.config.control_mode = CTRL_MODE_CURRENT_CONTROL

#     x_eq=0.11
#     y_eq=0.12

    kx = 100  # N/m
    ky = 200 # N/m
    kT = 0.0285 # Nm/A
    c  = 1  #1.5 # Nm/rad
    Imax=25    #A
    res = IK_5_link(x_eq, y_eq)
    L_eq,R_eq=enc_pos(res[0].x[0],res[0].x[1],cal_para)

    odrv0.axis0.controller.set_pos_setpoint(R_eq,0,0)
    odrv0.axis1.controller.set_pos_setpoint(L_eq,0,0)
    time.sleep(1)
    t_end = time.time() + 5

#     while time.time() < t_end:

    theta_l, theta_r = get_angles(odrv0,cal_para)
    vel = get_vel(odrv0)

    fk_xy = FK1_fast(theta_l,theta_r,link1_length,link2_length,w_length)
    J_xy = J1_fast(theta_l,theta_r,link1_length,link2_length,w_length)
    x = fk_xy[0]
    y = fk_xy[1]
    mat_xy = np.array([[x[0]-x_eq],
                       [y[0]-y_eq]])
    mat_k = np.array([[-kx,0],
                      [0,-ky]])
    I1 = (J_xy.T @ mat_k @ mat_xy)/kT
    I = I1 -c*(vel)
    if(I[0]>Imax):
        I[0]=Imax
    elif(I[0]<-Imax):
        I[0]=-Imax

    if(I[1]>Imax):
        I[1]=Imax
    elif(I[1]<-Imax):
        I[1]=-Imax

    odrv0.axis0.controller.set_current_setpoint(-I[1])
    odrv0.axis1.controller.set_current_setpoint(-I[0])
    print(I1)
    clear_output()
#     motor_idle(odrv0)    

# Sab yaha se start hoga. Baki upar ka aise he hai

In [370]:
def set_current(lim):
    JJ_odrv0.axis0.motor.config.current_lim = lim
    JJ_odrv0.axis1.motor.config.current_lim = lim

    SS_odrv0.axis0.motor.config.current_lim = lim
    SS_odrv0.axis1.motor.config.current_lim = lim

In [371]:
def set_pos_gain(gain):
    JJ_odrv0.axis0.controller.config.pos_gain = gain
    JJ_odrv0.axis1.controller.config.pos_gain = gain

    SS_odrv0.axis0.controller.config.pos_gain = gain
    SS_odrv0.axis1.controller.config.pos_gain = gain

In [409]:
def jump(JJ_odrv0, SS_odrv0):
#     height_offset = 600
#     ext_height = -600
#     rot_offset = 800 #500
#     land_offset = 600
    height_offset = 800
    ext_height = -1000
    rot_offset = 1000 #500
    land_offset = 1000
    
    # Using cal_para_JJ and cal_para_SS which are global variables
    JJ_zero_pos = (cal_para_JJ[3]+4096, cal_para_JJ[2])
    SS_zero_pos = (cal_para_SS[3]+4096, cal_para_SS[2])
    
    JJ_eq_pos = (JJ_zero_pos[0] - 3072 - height_offset - rot_offset, JJ_zero_pos[1] - 1024 + height_offset - rot_offset) #m0, m1
    JJ_crouch_pos = (JJ_zero_pos[0] - 3500, JJ_zero_pos[1] + 300)
    JJ_ext_pos = (JJ_crouch_pos[0] + (4096/2 + 200), JJ_crouch_pos[1] - 1000)
#     JJ_ext_pos = (JJ_crouch_pos[0] + (4096/2+400), JJ_crouch_pos[1]-2000 )
    JJ_landing = (JJ_zero_pos[0] - 4096 - land_offset, JJ_zero_pos[1] - land_offset)

    JJ_new_ext = (JJ_zero_pos[0] - 3072 - ext_height - rot_offset, JJ_zero_pos[1] - 1024 + ext_height - rot_offset)
    
    SS_eq_pos = (SS_zero_pos[0] - 3072 - height_offset + rot_offset, SS_zero_pos[1] - 1024 + height_offset + rot_offset) #m0, m1
    SS_crouch_pos = (SS_zero_pos[0] - 300 - 4096, SS_zero_pos[1] - 596)
    SS_ext_pos = (SS_crouch_pos[0] + 1000, SS_crouch_pos[1] - (4096/2 + 200))
#     SS_ext_pos = (SS_crouch_pos[0]+2000, SS_crouch_pos[1] - (4096/2+400))
    SS_landing = (SS_zero_pos[0] - 4086 + land_offset, SS_zero_pos[1] + land_offset)

    SS_new_ext = (SS_zero_pos[0] - 3072 - ext_height + rot_offset, SS_zero_pos[1] - 1024 + ext_height + rot_offset)
    set_current(20)
    set_pos_gain(60)

    # SET LEGS TO STARTING POS
    odrive.utils.dump_errors(JJ_odrv0, True)
    JJ_odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
    JJ_odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
    JJ_odrv0.axis0.controller.set_pos_setpoint(JJ_eq_pos[0],0,0)
    JJ_odrv0.axis1.controller.set_pos_setpoint(JJ_eq_pos[1],0,0)

    odrive.utils.dump_errors(SS_odrv0, True)
    SS_odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
    SS_odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
    SS_odrv0.axis0.controller.set_pos_setpoint(SS_eq_pos[0],0,0)
    SS_odrv0.axis1.controller.set_pos_setpoint(SS_eq_pos[1],0,0)

    time.sleep(3)

    # EXTEND
    set_current(20)
    set_pos_gain(13)
    JJ_odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
    JJ_odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
    JJ_odrv0.axis0.controller.set_pos_setpoint(JJ_ext_pos[0],0,0)
    JJ_odrv0.axis1.controller.set_pos_setpoint(JJ_ext_pos[1],0,0)

    SS_odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
    SS_odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
    SS_odrv0.axis0.controller.set_pos_setpoint(SS_ext_pos[0],0,0)
    SS_odrv0.axis1.controller.set_pos_setpoint(SS_ext_pos[1],0,0)

    time.sleep(0.25)

    # STARTING
    JJ_odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
    JJ_odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
    JJ_odrv0.axis0.controller.set_pos_setpoint(JJ_eq_pos[0],0,0)
    JJ_odrv0.axis1.controller.set_pos_setpoint(JJ_eq_pos[1],0,0)

    SS_odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
    SS_odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
    SS_odrv0.axis0.controller.set_pos_setpoint(SS_eq_pos[0],0,0)
    SS_odrv0.axis1.controller.set_pos_setpoint(SS_eq_pos[1],0,0)

    set_pos_gain(60)

In [348]:
# Right leg cycle (motor set SS)
def both_leg_Cycle(SS_odrv0,JJ_odrv0):
    SS_odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
    SS_odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
    JJ_odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
    JJ_odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
    
    start_time = time.time()
    a=np.load('R_leg1_200.npy')
#     a=w
    b=np.load('L_leg1_200.npy')
    b_top=b[0:51,:]
    b_bottom=b[51:101,:]
    w=np.block([[b_bottom],[b_top]])
    b=w
#     b=w
    # your code
    cpr= 2048
    t_step = 0.005
    w_ra = 5
    w_la = 5

    w_rb = 5
    w_lb = 5
#     for k in range(0,10):

    for i in range(0,len(a)):
        start_time2 = time.time()
        L_posJ=(np.rad2deg(b[i][1] )/cal_para_JJ[0])+cal_para_JJ[2]
        R_posJ=((np.rad2deg(b[i][0] )-180)/cal_para_JJ[1])+cal_para_JJ[3]
#         print(L_posJ)
        L_posS=(np.rad2deg(a[i][1] )/cal_para_SS[0])+cal_para_SS[2]
        R_posS=((np.rad2deg(a[i][0] )-180)/cal_para_SS[1])+cal_para_SS[3]

        if (i-1)>0:
            w_ra = ((b[i][0] - b[i-1][0])/(t_step*2*np.pi) )*cpr
            w_la = ((b[i][1] - b[i-1][1])/(t_step*2*np.pi) )*cpr

            w_rb = ((a[i][0] - a[i-1][0])/(t_step*2*np.pi) )*cpr
            w_lb = ((a[i][1] - a[i-1][1])/(t_step*2*np.pi) )*cpr

        SS_odrv0.axis0.controller.set_pos_setpoint(R_posS,1*w_rb,15)
        SS_odrv0.axis1.controller.set_pos_setpoint(L_posS,1*w_lb,15)
        

        JJ_odrv0.axis0.controller.set_pos_setpoint(R_posJ,1*w_ra,15)
        JJ_odrv0.axis1.controller.set_pos_setpoint(L_posJ,1*w_la,15)

        while (time.time() - start_time2 < 0.2*t_step):
            pass
# motor_idle()    
    elapsed_time = time.time() - start_time
#     print(elapsed_time)

In [365]:
JJ_odrv0.axis0.controller.config.pos_gain=120
SS_odrv0.axis0.controller.config.pos_gain=120

In [414]:
# Call each cycle for walking
N= 4 # No of steps
for i in range(4):
    both_leg_Cycle(SS_odrv0,JJ_odrv0)
# jump(JJ_odrv0,SS_odrv0) # uncomment this line to check jump sequence